# CS486 - Artificial Intelligence
## Lesson 27 - Sampling

Exact inference fails to scale for very large and complex Bayesian Networks. This section covers implementation of randomized sampling algorithms, also called Monte Carlo algorithms.

In [ ]:
import helpers
from aima.probability import *
from aima.notebook import psource

In [ ]:
psource(BayesNode.sample)

Before we consider the different algorithms in this section let us look at the **`BayesNode.sample`** method. It samples from the distribution for this variable conditioned on event's values for parent_variables. That is, return True/False at random according to with the conditional probability given the parents. The **`probability`** function is a simple helper from **`utils`** module which returns True with the probability passed to it.

### Prior Sampling

The idea of Prior Sampling is to sample from the Bayesian Network in a topological order. We start at the top of the network and sample as per **P(X<sub>i</sub> | parents(X<sub>i</sub>)** i.e. the probability distribution from which the value is sampled is conditioned on the values already assigned to the variable's parents. This can be thought of as a simulation.

In [ ]:
psource(prior_sample)

The function **`prior_sample`** implements the algorithm described in **Figure 14.13** of the book. Nodes are sampled in the topological order. The old value of the event is passed as evidence for parent values. We will use the Bayesian Network in **Figure 14.12** to try out the **`prior_sample`**

<center><img src="aima/images/sprinklernet.jpg" height="500" width="500"></center>

Traversing the graph in topological order is important.
There are two possible topological orderings for this particular directed acyclic graph.

1. `Cloudy -> Sprinkler -> Rain -> Wet Grass`
2. `Cloudy -> Rain -> Sprinkler -> Wet Grass`

We can follow any of the two orderings to sample from the network.
Any ordering other than these two, however, cannot be used.

One way to think about this is that `Cloudy` can be seen as a precondition of both `Rain` and `Sprinkler` and just like we have seen in planning, preconditions need to be satisfied before a certain action can be executed.

We store the samples on the observations. Let us find **P(Rain=True)** by taking 1000 random samples from the network.

In [ ]:
N = 1000
all_observations = [prior_sample(sprinkler) for x in range(N)]

Now we filter to get the observations where Rain = True

In [ ]:
rain_true = [observation for observation in all_observations if observation['Rain'] == True]

Finally, we can find $P(Rain=True)$

In [ ]:
len(rain_true) / N

Sampling this another time might give different results as we have no control over the distribution of the random samples

In [ ]:
N = 1000
all_observations = [prior_sample(sprinkler) for x in range(N)]
rain_true = [observation for observation in all_observations if observation['Rain'] == True]
len(rain_true) / N

To evaluate a conditional distribution. We can use a two-step filtering process. We first separate out the variables that are consistent with the evidence. Then for each value of query variable, we can find probabilities. For example to find **P(Cloudy=True | Rain=True)**. We have already filtered out the values consistent with our evidence in **rain_true**. Now we apply a second filtering step on **rain_true** to find **P(Rain=True and Cloudy=True)**

In [ ]:
rain_and_cloudy = [observation for observation in rain_true if observation['Cloudy'] == True]
len(rain_and_cloudy) / len(rain_true)

### Rejection Sampling

Rejection Sampling is based on an idea similar to what we did just now. First, it generates samples from the prior distribution specified by the network. Then, it rejects all those that do not match the evidence. Rejection sampling is advantageous only when we know the query beforehand. While prior sampling generally works for any query, it might fail in some scenarios.

Let's say we have a generic Bayesian network and we have evidence `e`, and we want to know how many times a state `A` is true, given evidence `e` is true. Normally, prior sampling can answer this question, but let's assume that the probability of evidence `e` being true in our actual probability distribution is very small. In this situation, it might be possible that sampling never encounters a data-point where `e` is true. If our sampled data has no instance of `e` being true, `P(e) = 0`, and therefore `P(A | e) / P(e) = 0/0`, which is undefined. We cannot find the required value using this sample. 

We can definitely increase the number of sample points, but we can never guarantee that we will encounter the case where `e` is non-zero (assuming our actual probability distribution has at least one case where `e` is true).
To guarantee this, we would have to consider every single data point, which means we lose the speed advantage that approximation provides us and we essentially have to calculate the exact inference model of the Bayesian network. Rejection sampling will be useful in this situation, as we already know the query.

While sampling from the network, we will reject any sample which is inconsistent with the evidence variables of the given query (in this example, the only evidence variable is `e`).
We will only consider samples that do not violate **any** of the evidence variables.
In this way, we will have enough data with the required evidence to infer queries involving a subset of that evidence.

The function **rejection_sampling** implements the algorithm described by **Figure 14.14**

In [ ]:
psource(rejection_sampling)

The function keeps counts of each of the possible values of the Query variable and increases the count when we see an observation consistent with the evidence. It takes the query variable, evidence, the Bayes net and the number of prior samples to generate.

**`consistent_with`** is used to check consistency.

In [ ]:
psource(consistent_with)

To answer **P(Cloudy=True | Rain=True)**

In [ ]:
rejection_sampling('Cloudy', {'Rain': True}, sprinkler)

### Likelihood Weighting

Rejection sampling takes a long time to run when the probability of finding consistent evidence is low. It is also slow for larger networks and more evidence variables.
Rejection sampling tends to reject a lot of samples if our evidence consists of a large number of variables. Likelihood Weighting solves this by fixing the evidence (i.e. not sampling it) and then using weights to make sure that our overall sampling is still consistent.

The pseudocode in **Figure 14.15** is implemented as **`likelihood_weighting`** and **`weighted_sample`**.

In [ ]:
psource(weighted_sample)


**`weighted_sample`** samples an event from Bayesian Network that's consistent with the evidence **e** and returns the event and its weight, the likelihood that the event accords to the evidence. It takes in two parameters **bn** the Bayesian Network and **e** the evidence.

The weight is obtained by multiplying $P(x_i\mid{parents(x_i)})$ for each node in evidence. We set the values of **event = evidence** at the start of the function.

In [ ]:
weighted_sample(sprinkler, {'Rain': True})

In [ ]:
psource(likelihood_weighting)

**`likelihood_weighting`** implements the algorithm to solve our inference problem. The code is similar to **`rejection_sampling`** but instead of adding one for each sample we add the weight obtained from **`weighted_sampling`**.

In [ ]:
likelihood_weighting('Cloudy', {'Rain': True}, sprinkler, 200)

### Gibbs Sampling

In likelihood sampling, it is possible to obtain low weights in cases where the evidence variables reside at the bottom of the Bayesian Network. This can happen because influence only propagates downwards in likelihood sampling.

Gibbs Sampling solves this. The implementation of **Figure 14.16** is provided in the function **gibbs_ask** 

In [ ]:
psource(gibbs_ask)
psource(markov_blanket_sample)

In **`gibbs_ask`** we initialize the non-evidence variables to random values. And then select non-evidence variables and sample it from **P(Variable | value in the current state of all remaining vars) ** repeatedly sample. In practice, we speed this up by using **`markov_blanket_sample`** instead. This works because terms not involving the variable get canceled in the calculation. The arguments for **`gibbs_ask`** are similar to **`likelihood_weighting`**

In [ ]:
gibbs_ask('Cloudy', {'Rain': True}, sprinkler, 200)

#### Runtime analysis
Let's take a look at how much time each algorithm takes.

In [ ]:
%%timeit
all_observations = [prior_sample(sprinkler) for x in range(1000)]
rain_true = [observation for observation in all_observations if observation['Rain'] == True]
len([observation for observation in rain_true if observation['Cloudy'] == True]) / len(rain_true)

In [ ]:
%%timeit
rejection_sampling('Cloudy', {'Rain': True}, sprinkler, 1000)

In [ ]:
%%timeit
likelihood_weighting('Cloudy', {'Rain': True}, sprinkler, 200)

In [ ]:
%%timeit
gibbs_ask('Cloudy', {'Rain': True}, sprinkler, 200)

As expected, all algorithms have a very similar runtime. However, rejection sampling would be a lot faster and more accurate when the probability of finding data-points consistent with the required evidence is small. Likelihood weighting is the fastest out of all as it doesn't involve rejecting samples, but also has a quite high variance.